In [9]:
%matplotlib inline
import pandas as pd
import numpy as np
import artpop
import astropy.units as u
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.visualization import make_lupton_rgb
from random import gauss

In [4]:
artpop.filters.phot_system_lookup()

{'UKIDSS_Z': 'UKIDSS',
 'UKIDSS_Y': 'UKIDSS',
 'UKIDSS_J': 'UKIDSS',
 'UKIDSS_H': 'UKIDSS',
 'UKIDSS_K': 'UKIDSS',
 'R062': 'WFIRST',
 'Z087': 'WFIRST',
 'Y106': 'WFIRST',
 'J129': 'WFIRST',
 'W146': 'WFIRST',
 'H158': 'WFIRST',
 'F184': 'WFIRST',
 'LSST_u': 'LSST',
 'LSST_g': 'LSST',
 'LSST_r': 'LSST',
 'LSST_i': 'LSST',
 'LSST_z': 'LSST',
 'LSST_y': 'LSST',
 'DECam_u': 'DECam',
 'DECam_g': 'DECam',
 'DECam_r': 'DECam',
 'DECam_i': 'DECam',
 'DECam_z': 'DECam',
 'DECam_Y': 'DECam',
 'SDSS_u': 'SDSSugriz',
 'SDSS_g': 'SDSSugriz',
 'SDSS_r': 'SDSSugriz',
 'SDSS_i': 'SDSSugriz',
 'SDSS_z': 'SDSSugriz',
 'WFC3_UVIS_F200LP': 'HST_WFC3',
 'WFC3_UVIS_F218W': 'HST_WFC3',
 'WFC3_UVIS_F225W': 'HST_WFC3',
 'WFC3_UVIS_F275W': 'HST_WFC3',
 'WFC3_UVIS_F280N': 'HST_WFC3',
 'WFC3_UVIS_F300X': 'HST_WFC3',
 'WFC3_UVIS_F336W': 'HST_WFC3',
 'WFC3_UVIS_F343N': 'HST_WFC3',
 'WFC3_UVIS_F350LP': 'HST_WFC3',
 'WFC3_UVIS_F373N': 'HST_WFC3',
 'WFC3_UVIS_F390M': 'HST_WFC3',
 'WFC3_UVIS_F390W': 'HST_WFC3',
 'WFC3

In [7]:
ssp = artpop.MISTSSP(
    log_age = 10.1,       # log of age in years
    feh = -1.5,           # metallicity [Fe/H]
    phot_system = 'HST_ACSWF', # photometric system(s)
    num_stars = 5e5,      # number of stars
#     random_state = rng,   # random state for reproducibility
)

In [8]:
ssp.abs_mag_table

ACS_WFC_F435W,ACS_WFC_F475W,ACS_WFC_F502N,ACS_WFC_F550M,ACS_WFC_F555W,ACS_WFC_F606W,ACS_WFC_F625W,ACS_WFC_F658N,ACS_WFC_F660N,ACS_WFC_F775W,ACS_WFC_F814W,ACS_WFC_F850LP,ACS_WFC_F892N
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
10.366155713384455,9.875262739228601,9.73232096040524,8.983384077854879,9.223930426766698,8.77637880102723,8.478763029050835,8.149305882318474,8.23536001296735,7.835888084556086,7.7367110307697535,7.489464751378081,7.521219703069468
12.358723035678677,11.803833816529044,11.724279812654833,10.77022937432018,11.056210348801152,10.533477163487158,10.204326417325728,9.845419593825696,9.931062893939435,9.478399143619704,9.365018882850626,9.086794060736695,9.119156614937713
12.21657786661951,11.667353203621158,11.583275397250178,10.644764541097528,10.92738709439514,10.4100712278479,10.08296808556035,9.726867163149,9.812654477951387,9.36394586636232,9.251721262665644,8.975894017288251,9.008161140753327
11.878133987567873,11.338458592835181,11.245423616279322,10.33568546656326,10.61199569992376,10.104354777740136,9.781033816284069,9.429640471662818,9.515641321438752,9.073573793568439,8.96327283732218,8.691461547602797,8.723650320599994
11.429849964231094,10.900595913052303,10.79715720068794,9.920227808972387,10.189292238047491,9.692846685576948,9.374156061750924,9.027703109034189,9.113867766815613,8.679449586664047,8.571297057855398,8.304087960254515,8.336262026032658
11.93082257043977,11.389909087824876,11.298149862222855,10.384470785647242,10.661650008787893,10.15272306390362,9.8288707107459,9.476863115665392,9.56283862846586,9.119911697779084,9.009362850043138,8.737033179501394,8.769224691631074
14.105225758245322,13.45488810497126,13.48755961230949,12.231742084378888,12.583372932885883,11.984440887106858,11.630586564722444,11.178988082560972,11.259508132143083,10.753518600040762,10.620566535551113,10.302633907591366,10.336289275268411
14.387392285544376,13.719076648229638,13.786213267747778,12.459362528948368,12.826753097404854,12.214403689157622,11.858021057527742,11.375313820962875,11.45413285029408,10.941649552368322,10.803856847492876,10.47891002490132,10.512417004385966
12.533215815709625,11.971170441964286,11.897477355874368,10.923688891674445,11.213857017141217,10.684557829121072,10.35293433827915,9.99020619058115,10.07564245465568,9.618149945362815,9.503317420093968,9.222137809140454,9.25461980896044


In [ ]:
def perturb_mags(model_filts, input_colors, input_mags, input_color_err, input_mag_err):
    """
    Make model CMD more realistic by 1. matching the limiting magnitude and 2. perturbing ArtPop stellar magnitudes by the errors on the input CMD.
    
    PARAMETERS:
        model_filts (str, arr-like): list of filters used in CMD -- [bluer filter, redder filter], e.g. ["ACS_WFC_F606W", "ACS_WFC_F814W"]
        input_colors (arr-like): x-axis of data-based CMD, e.g. F606W-F814W
        input_mags (arr-like): y-axis of data-based CMD, e.g. F814W
        input_color_errs (arr-like): error on x-axis of data-based CMD
        input_mag_errs (arr-like): error y-axis of data-based CMD
    
    
    RETURNS: 
        CMD x-axis (perturbed model colors, array), CMD y-axis (perturbed model magnitudes, array) 
    """
    
    pop = ssp.star_mags(model_filts[1])
    pop_color = ssp.star_mags(model_filts[0]) - ssp.star_mags(model_filts[1])

    pop_err = []
    pop_color_err = []
    new_pop = []
    new_popcolor = []

    for i in range(len(pop)):
        pop_err_mod = input_mag_err[np.argmin(abs(input_mags - pop[i]))]
        pop_err.append(pop_err_mod)
        new_pop.append(pop[i] + gauss(0, pop_err**2))

        pop_color_err_mod = input_mag_err[np.argmin(abs(input_mags - pop[i]))]
        pop_color_err.append(pop_color_err_mod)
        new_popcolor.append(pop_color[i] + gauss(0, pop_color_err**2))
        
    return new_popcolor, new_pop

In [ ]:
def plot_results(img_filts, reff, n, theta, ellip, distance, pixel_scale, CMD_filts = img_filts[-2:], zpts = None, psfs = None, xy_dim = 1001, Q = 0.5, stretch = 0.06, save = True)
    stars = ssp.abs_mag_table

    xy = artpop.sersic_xy(
        num_stars = len(stars),     
        r_eff = reff*u.kpc,         # effective radius (kpc)
        n = n,             # Sersic index
        theta = theta*u.deg,          # position angle (deg)
        ellip = ellip,         # ellipticity
        distance = distance*u.Mpc,       # distance to system (Mpc)
        xy_dim = xy_dim, # xy dimensions of image
        pixel_scale = pixel_scale
    )
    
    distance_mod = 5*np.log10(distance*u.Mpc.to(u.pc)/10)
    
    mags_ = {'ACS_WFC_F475W': stars['WFC_ACS_F475W'] + distance_mod, 'ACS_WFC_F606W': stars['WFC_ACS_F606W'] + distance_mod, 
         'ACS_WFC_F814W': stars['WFC_ACS_F814W'] + distance_mod} #might need to convert names/between abs mag and app mag; will need to see as we work on it
    
    ssp = artpop.source.Source(xy, mags = mags_, xy_dim = 1001, pixel_scale = 0.05)
    
    img = {}
    filt = img_filts
    zpt_ = zpts
    psf_ = psfs
    for i in range(len(filt)):
        img[filt[i]] = imager.observe(ssp, bandpass = img_filt[i], psf = psf_[i], zpt = zpt_[i]).image
    
    if (len(img_filts) > 3) or (len(img_filts) < 2):
        raise Exception("Must have two or three filters to generate mock image.")
    
    if len(img_filts) == 2:
        b = img[img_filts[0]]
        g = (img[img_filts[0]] + img[img_filts[1]])/2
        r = img[img_filts[1]]
        
    else:
        b = img[img_filts[0]]
        g = img[img_filts[1]]
        r = img[img_filts[2]]

    rgb = make_lupton_rgb(r, g, b, Q=Q, stretch=stretch)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))
    ax1.scatter(mags_[CMD_filts[0]] - mags_[CMD_filts[1]], mags_[CMD_filts[1]], color = 'k', s = 10)
    ax1.set_xlabel(CMD_filts[0] + '-' + CMD_filts[1])
    ax1.set_ylabel(CMD_filts[1])
    
    ax2.imshow(rgb, origin = 'lower')
    ax2.axis('off')
    
    if save == True:
        fig.savefig('FitAP_out.pdf', bbox_inches = 'tight', dpi = 200)